In [ ]:
pip install transformers

In [ ]:
!nvidia-smi

Wed Feb 21 13:30:26 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import torch

In [ ]:
pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.6 MB/s eta 0:00:00


In [ ]:
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer

In [ ]:
model_path = "LR-AI-Labs/vbd-llama2-7B-50b-chat"

tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    model_path, torch_dtype=torch.bfloat16,
    device_map='auto',
)
model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/766 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/795k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/3.82G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(49381, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
 

In [ ]:
def response_generate(input_prompt):
    input_ids = tokenizer(input_prompt, return_tensors="pt")
    outputs = model.generate(
        inputs=input_ids["input_ids"].to("cuda"),
        attention_mask=input_ids["attention_mask"].to("cuda"),
        do_sample=True,
        temperature=0.1,
        top_k=5,
        top_p=0.9,
        max_new_tokens=1024,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id
    )
    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    response = response.split("ASSISTANT:")[-1].strip()
    return response


In [ ]:
# SYS_PROMPT = """Trong vai trợ lý phân tích ý định, hãy phân tích và chia nhỏ đoạn lệnh thành các câu lệnh đơn (nếu có) để phần mềm có thể thực hiện tuần tự: "Hãy chọn tất cả các hình chữ nhật có trong vùng chọn được xác định bởi 2 tọa độ (123, 124) và (43, 13) rồi dịch chúng qua bên trái 14 pixel".""" \

# """Bạn có thể tham khảo các ví dụ dưới đây để có thể tách được câu được yêu cầu:""" \
# """- Đầu vào: "vẽ hình tròn màu đen ở vị trí (20, 40) rồi tô màu đen cho nó". => Đầu ra: ["vẽ hình tròn màu đen ở vị trí (20, 40)", "tô màu đen cho hình tròn ở vị trí (20, 40)"]""" \
# """- Đầu vào : "vẽ hình vuông và hình tròn với cùng kích thước 40 x 40 tại 2 vị trí là (24,14) và góc trên bên trái màn hình" => Đầu ra: ["vẽ hình vuông có kích thước 40 x 40 tại vị trí (24, 14)", "Vẽ hình tròn có kích thước 40 x 40 tại vị trí góc trên bên trái màn hình"].""" \
# """- Đầu vào: "xoay hình vuông có cạnh dài 50 pixel sang phải 15 độ rồi tô màu đen cho nó  => Đầu ra: ["xoay hình vuông có cạnh dài 50 pixel sang phải 15 độ", "tô màu đen cho hình vuông có cạnh dài 50 pixel"].""" \
# """Chú ý: bạn chỉ cần đưa ra kết quả có bố cục tương tự đầu ra trong các ví dụ trên, không cần đưa ra thông tin nào khác."""
paragraph = "Chọn các hình có trong vùng được xác định bởi 2 tọa độ là (123, 134) và (1234, 2141) sau đó dịch chuyển chúng sang phải 13 pixel"
SYS_PROMPT= "A chat between a curious user and an artificial intelligence assistant. "\
        "The assistant gives helpful, detailed, and polite answers to the user's questions based on previous responses of assistant."
print(response_generate(f"{SYS_PROMPT}" \
                        f"""USER: Trong vai trợ lý phân tích ý định, hãy phân tích và chia nhỏ đoạn lệnh thành các câu lệnh đơn (nếu có) để phần mềm có thể thực hiện tuần tự: "Hãy vẽ hình vuông có kích thước cạnh bằng 50 pixel ở vị trí chính giữa màn hình rồi nghiêng nó sang phải khoảng 15 độ.".""" \
                        f"""Bạn có thể tham khảo các ví dụ dưới đây để có thể tách được câu được yêu cầu:""" \
                        f"""- Đầu vào: "vẽ hình tròn màu đen ở vị trí (20, 40) rồi tô màu đen cho nó". => Đầu ra: ["vẽ hình tròn màu đen ở vị trí (20, 40)", "tô màu đen cho hình tròn ở vị trí (20, 40)"]""" \
                        f"""- Đầu vào : "vẽ hình vuông và hình tròn với cùng kích thước 40 x 40 tại 2 vị trí là (24,14) và góc trên bên trái màn hình" => Đầu ra: ["vẽ hình vuông có kích thước 40 x 40 tại vị trí (24, 14)", "Vẽ hình tròn có kích thước 40 x 40 tại vị trí góc trên bên trái màn hình"].""" \
                        f"""- Đầu vào: "xoay hình vuông có cạnh dài 50 pixel sang phải 15 độ rồi tô màu đen cho nó  => Đầu ra: ["xoay hình vuông có cạnh dài 50 pixel sang phải 15 độ", "tô màu đen cho hình vuông có cạnh dài 50 pixel"].""" \
                        f"""Chú ý: bạn chỉ cần đưa ra kết quả có bố cục tương tự đầu ra trong các ví dụ trên, không cần đưa ra thông tin nào khác. Đồng thời hãy để ý đến các quan hệ từ và xác định vai trò của chúng trong việc tách câu. """ \
                        f"""ASSISTANT:""" \
                        f"""- Vẽ hình vuông có kích thước cạnh bằng 50 pixel ở vị trí chính giữa màn hình. \n""" \
                        f"""- Nghiêng hình vuông sang phải khoảng 15 độ.""" \
                        f"""USER: Vẽ hình mũi tên có chiều rộng và chiều cao lần lượt là 70 và 30 ở chính giữa sau đó tạo bản sao của nó ở góc bên trái. """ \
                        f"""ASSISTANT:"""
                        f"""- Vẽ hình mũi tên có chiều rộng và chiều cao lần lượt là 70 và 30 ở vị trí chính giữa \n""" \
                        f"""- Sao chép \n""" \
                        f"""- Dán vào góc bên trái""" \
                        f"""USER: Tạo hình bình hành màu xanh dương , có chiều dài đáy là 25 đơn vị , chiều cao là 35 đơn vị , nằm ở vị trí tương đối trên cùng bên phải . """ \
                        f"""ASSISTANT:"""
                        f""" Tạo hình bình hành màu xanh dương có chiều dài đáy là 25 đơn vị và chiều cao là 35 đơn vị , nằm ở vị trí tương đối trên cùng bên phải .""" \
                        f"""USER: {paragraph}""" \
                        f"""ASSISTANT:"""))

# Xin chào, ViVi là một trợ lý trí tuệ nhân tạo có thể trả lời câu hỏi của bạn và trò chuyện với bạn.


Dịch chuyển các hình đã chọn sang phải 13 pixel.
